In [1]:
import pandas as pd

df = pd.read_csv('../AI데이터/fruits.csv')
df

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


# 결측값 확인

In [3]:
df.isnull().sum()

name        0
diameter    0
weight      0
red         0
green       0
blue        0
dtype: int64

In [4]:
df2 = df.copy()
df2

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


# 종속변수인 name 레이블 인코딩 전처리 진행 

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(df.iloc[:, 0])
df2['name'] = labels

df2

,name,diameter,weight,red,green,blue
0,1,2.96,86.76,172,85,2
1,1,3.91,88.05,166,78,3
2,1,4.42,95.17,156,81,2
3,1,4.47,95.60,163,81,4
4,1,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,0,15.35,253.89,149,77,20
9996,0,15.41,254.67,148,68,7
9997,0,15.59,256.50,168,82,20
9998,0,15.92,260.14,142,72,11


In [8]:
df2.iloc[:, 1:]

,diameter,weight,red,green,blue
0,2.96,86.76,172,85,2
1,3.91,88.05,166,78,3
2,4.42,95.17,156,81,2
3,4.47,95.60,163,81,4
4,4.48,95.76,161,72,9
...,...,...,...,...,...
9995,15.35,253.89,149,77,20
9996,15.41,254.67,148,68,7
9997,15.59,256.50,168,82,20
9998,15.92,260.14,142,72,11


In [9]:
df2.iloc[:, 0] 

0       1
1       1
2       1
3       1
4       1
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: name, Length: 10000, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df2.iloc[:, 1:], 
    df2.iloc[:, 0], 
    test_size =0.2, 
    random_state = 42
)

In [11]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 2)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [12]:
score = model.score(X_test, y_test)
print('정확도 : ', score)

정확도 :  0.8885


In [13]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 1, 0], shape=(2000,))

In [14]:
import pandas as pd

comparison = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

print(comparison.head(10))  # 앞에서 10개만 미리보기

   Actual  Predicted
0       0          0
1       1          0
2       1          1
3       1          0
4       1          0
5       0          0
6       1          1
7       0          1
8       0          0
9       1          1


# GridSearchCV 최적의 하이퍼 파라미터 찾기

In [15]:
from sklearn.model_selection import GridSearchCV, KFold

estimator = DecisionTreeClassifier()
cv = KFold(n_splits=5, random_state=42, shuffle=True)

parameters = {'criterion' : ['gini', 'entropy'],
            'max_depth' : [2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_samples_split' : [2, 4, 6, 8, 10, 12, 14, 16],
            'max_leaf_nodes' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_samples_leaf' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'splitter' : ['best', 'random'],
            'random_state' : [7, 23, 42, 78]}

model = GridSearchCV(estimator = estimator,
                    param_grid = parameters,
                    cv = cv, verbose = 1,
                    n_jobs = -1, refit = True) # -1 현재 시스템에서 병렬개수를 최대한 사용하겠다

model.fit(X_train, y_train)

Fitting 5 folds for each of 115200 candidates, totalling 576000 fits


C:\Project\Python_Source\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
57600 fits failed out of a total of 576000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
57600 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Project\Python_Source\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Project\Python_Source\.venv\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "C:\Project\Python_Source\.venv\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "C:\Projec

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_leaf_nodes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16],
                         'random_state': [7, 23, 42, 78],
                         'splitter': ['best', 'random']},
             verbose=1)

In [16]:
import warnings
warnings.filterwarnings(action='ignore')

print("Best Estimator:\n", model.best_estimator_);print()
print("Best Params:\n", model.best_params_);print()
print("Best Score:\n", model.best_score_);print()

Best Estimator:
 DecisionTreeClassifier(criterion='entropy', max_depth=4, max_leaf_nodes=9,
                       random_state=7)

Best Params:
 {'criterion': 'entropy', 'max_depth': 4, 'max_leaf_nodes': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 7, 'splitter': 'best'}

Best Score:
 0.912625



In [17]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=4,
    max_leaf_nodes=9,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=7,
    splitter='best'
)

model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, max_leaf_nodes=9,
                       random_state=7)

In [18]:
score = model.score(X_test, y_test)
print('정확도 : ', score)

정확도 :  0.9155


In [19]:
import pandas as pd

comparison = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

print(comparison.tail(30))  # 앞에서 10개만 미리보기

      Actual  Predicted
1970       1          0
1971       1          1
1972       1          1
1973       1          1
1974       1          1
1975       0          1
1976       1          1
1977       1          0
1978       0          1
1979       1          1
1980       1          1
1981       1          1
1982       1          1
1983       1          1
1984       1          1
1985       0          0
1986       1          1
1987       0          0
1988       0          0
1989       1          1
1990       1          1
1991       0          0
1992       0          0
1993       1          1
1994       0          0
1995       0          0
1996       0          0
1997       0          0
1998       1          1
1999       0          0
